## Our 4 Starting Question we decided to change to a single one:


1. How well can the level of corruption in a European country be quantified?

This questions is too broad and may lack specificity. Also not a good question to answer within a notebook.
The definition of "quantified" needs clarification—are we looking at an index, a model, or a metric?

2. Are there different forms of corruption prevalent in different European countries?

While interesting, it may be challenging to obtain granular, comparable data across countries.
Requires deep qualitative insights, which may not be fully captured by available datasets.

3. What characteristics of a country predict the level of corruption?

Why it's promising:

Allows for quantitative analysis using regression or classification models.
Can leverage socio-economic, political, and governance indicators.
Provides actionable insights for policymakers and organizations.
Well-defined and measurable through publicly available datasets.
Potential Challenges:

Ensuring data quality and avoiding biases in reporting.
Distinguishing correlation from causation.

4. What characteristics of a country predict an increase or decrease in the level of corruption?

Why it's promising:

Focuses on change over time, enabling trend analysis.
Useful for policy evaluation and forecasting.
Encourages a deeper exploration of temporal datasets (e.g., economic reforms, governance improvements, etc.).
Can help identify early warning signs for rising corruption.
Potential Challenges:

Requires time-series data and careful handling of lag effects.
External factors (global economic crises, political events) may introduce noise.


The following question seems the most promising and we decided to only choose this one:

# What characteristics of a european country predict the level of corruption?

Why it's promising:

Allows for quantitative analysis using regression or classification models.
Can leverage socio-economic, political, and governance indicators.
Provides actionable insights for policymakers and organizations.
Well-defined and measurable through publicly available datasets.
Potential Challenges:

Ensuring data quality and avoiding biases in reporting.
Distinguishing correlation from causation.

maybe we could tackle this one but lets find out later: Which country characteristics (e.g., economic, political, social indicators) best predict the level of corruption in European countries, and how do these characteristics relate to any gap between actual and perceived corruption?



# Start

As we have decided to focus on the continent of europe the first challenge will be to create a dataset of all european countries. To make this notebook not substantially long we will provide the finished data **europe_countries** to you. It shows all european countries and their ISO3 Code and ISO2 Code which should allow better preprocessing later.

In [1]:
import numpy as np
import pandas as pd

In [238]:
countries = pd.read_csv("../data/processed/europe_countries.csv")

In [239]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    49 non-null     object
 1   ISO3 Code  49 non-null     object
 2   ISO2 Code  49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [240]:
countries['Country'].unique()

array(['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium',
       'Bosnia & Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
       'Italy', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania',
       'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro',
       'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia',
       'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
       'United Kingdom', 'Vatican City'], dtype=object)

When looking at european countrie numbers, many figures fly arround, our data includes 48 countries, and of course the most promising ones. As we will see later most of the other datasets aint serve data for all countries anyway. Especially for small countries like Vatican or Monaco, etc.

### Corruption Perceptions Index (CPI) from Transparency International.
Data Set that shows preceived corruption of countries and rank them.

Link to data: https://images.transparencycdn.org/images/CPI2023_FullDataSet.zip
https://www.transparency.org/en/news/how-cpi-scores-are-calculated

 0   Economy ISO3                                
 1   Economy Name                                 
 2   Year                                           
 3   Corruption Perceptions Index Rank            
 4   Corruption Perceptions Index Score           
 5   Corruption Perceptions Index Sources        
 6   Corruption Perceptions Index Standard Error

We took the xlsx and transformed it to this more comapct version with only features we need and only european countries. Data points get explained at the end of each processing.

In [5]:
cpi_data = pd.read_csv("../data/processed/CPI.csv")

In [13]:
iso3_europe_full = set(countries["ISO3 Code"])

iso3_europe_cpi = set(cpi_data["Economy ISO3"])

print('Lenght of all europe countries in our list: ',len(iso3_europe_full))
print('Lenght of cpi countriest:', len(iso3_europe_cpi))

print('Countries mssing')

iso3_europe_full-iso3_europe_cpi

Lenght of all europe countries in our list:  48
Lenght of cpi countriest: 42
Countries mssing


{'AND', 'LIE', 'MCO', 'RKS', 'SMR', 'VAT'}

Countries missing: Andora, Liechtenstein, Kosovo, San Marino, Vatikan, Gibraltar, Azerbaijan. We could neclect this countries due to their small size and impact.

In [21]:
cpi_data['Year'].unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
       2023])

In [23]:
cpi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 7 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Economy ISO3                                 504 non-null    object 
 1   Economy Name                                 504 non-null    object 
 2   Year                                         504 non-null    int64  
 3   Corruption Perceptions Index Rank            491 non-null    float64
 4   Corruption Perceptions Index Score           504 non-null    float64
 5   Corruption Perceptions Index Sources         504 non-null    float64
 6   Corruption Perceptions Index Standard Error  504 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 27.7+ KB


#### CPI Dataset Summary
* Corruption Perceptions Index Rank (float64):
This represents the ranking of the country based on its Corruption Perceptions Index (CPI) score, with rank 1 being the least corrupt country.
Lower ranks indicate better perceived transparency.

* Corruption Perceptions Index Score (float64):
This score quantifies perceived corruption levels on a scale of 0 to 100, where:
0 = highly corrupt
100 = very clean (low corruption perception)
It is calculated using multiple data sources, standardized, and aggregated.

* Corruption Perceptions Index Sources (float64):
The number of sources used to calculate the CPI score for a given country.
The CPI uses multiple expert assessments and business surveys; having more sources increases reliability.

*  Corruption Perceptions Index Standard Error (float64):
This represents the uncertainty or variability in the CPI score, showing how much variation exists among different data sources.
A lower standard error indicates a more reliable score, while a higher one suggests more disagreement or variability in corruption perceptions across sources.

### GDP Data

We decided to go for GDP per Capita (PPP - Purchasing Power Parity)

Why?

Adjusted for cost of living, making it a fairer comparison across countries.
Helps analyze the standard of living and economic well-being in relation to corruption.
Frequently used in corruption-related studies to measure economic development.

Definition: A country's gross domestic product (GDP) at purchasing power parity (PPP) per capita is the PPP value of all final goods and services produced within an economy in a given year, divided by the average (or mid-year) population for the same year. This is similar to nominal GDP per capita but adjusted for the cost of living in each country.



## GDP Per Capita

### 1. Basic Information
- **Data Source**: https://databank.worldbank.org/indicator/NY.GDP.PCAP.CD/1ff4a498/Popular-Indicators?l=en#advancedDownloadOptions
- **Description**: The dataset contains GDP per capita data (current US$) for European countries from 2012 to 2023.
- **Year/Coverage**: 2012-2023

### 2. Key Variables / Columns

| **Column Name**   | **Type** | **Description**                       |
|------------------|----------|---------------------------------------|
| Country Name     | object   | Name of the country                   |
| Country Code     | object   | ISO 3 code of the country               |
| Year             | object   | Year of the recorded GDP value        |
| GDP_per_capita   | float64  | GDP per capita in current US dollars  |

### 3. Data Cleaning / Transformation
- **Original Format**: Wide format with years as columns.
- **Filtering**: Retained only European countries.
- **Columns Kept/Removed**: Dropped `Series Code` and `Series Name` columns.
- **Data Type Conversions**: Converted `Year` to string and `GDP_per_capita` to float.
- **Handling Missing Values**: -

### 4. Data Context & Usage
- **Why This Dataset**: To analyze economic trends across European countries by measuring GDP per capita over time.
- **Potential Biases**: -
- **Reliability Considerations**: The data's reliability depends on the accuracy of the original source.

### 5. Additional Notes / References
- **Version / Date of Retrieval**: 20.01.25



In [88]:
gdp_ppp = pd.read_csv("../data/raw/gdp_per_capita.csv")

In [71]:

gdp_ppp =gdp_ppp.drop(labels=["Series Code", "Series Name"], axis= 1)

gdp_ppp = gdp_ppp.drop(gdp_ppp.tail(8).index)

gdp_ppp.columns = gdp_ppp.columns.str.replace(r"\[.*?\]", "", regex=True).str.strip()


gdp_ppp.head()


,Country Name,Country Code,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Albania,ALB,4247.631343,4413.063383,4578.633208,3952.803574,4124.055390,4531.032207,5365.489347,5460.428237,5370.778623,6413.283286,6846.426143,8575.171134
1,Andorra,AND,41500.543579,42470.316116,44369.659691,38654.934720,40129.819201,40672.994335,42819.774580,41257.804585,37361.090067,42425.699676,42414.059009,46812.448449
2,Austria,AUT,48250.405914,50305.354577,51314.972262,43915.228021,45061.499392,47163.742578,51194.074984,49885.994736,48716.409890,53648.719074,52176.664914,56033.573792
3,Belarus,BLR,6953.215917,7998.080205,8341.290143,5967.068560,5039.775609,5785.533977,6360.053101,6837.768321,6542.864540,7489.718947,7994.648061,7829.053137
4,Belgium,BEL,44874.170918,46964.594678,47995.778696,40893.804538,41854.549830,44035.323936,47487.210039,46716.622747,45906.287581,51655.788330,50807.204708,54700.909324


In [76]:
# Convert wide format to long format
gdp_ppp_long = gdp_ppp.melt(
    id_vars=['Country Name', 'Country Code'],
    var_name='Year',
    value_name='GDP_per_capita'
)

In [83]:
#gdp_ppp_long.sort_values(["Country Name", "Year"]).head()

In [87]:
gdp_ppp_long.to_csv("../data/processed/gdp_per_capita.csv")

In [90]:
gdp_ppp_long.head()

,Country Name,Country Code,Year,GDP_per_capita
0,Albania,ALB,2012,4247.631343
1,Andorra,AND,2012,41500.543579
2,Austria,AUT,2012,48250.405914
3,Belarus,BLR,2012,6953.215917
4,Belgium,BEL,2012,44874.170918


### FDI Data

Function to filter the datasets on our european country list.

Foreign direct investment, net inflows (% of GDP)
Long definition
Foreign direct investment are the net inflows of investment to acquire a lasting management interest (10 percent or more of voting stock) in an enterprise operating in an economy other than that of the investor. It is the sum of equity capital, reinvestment of earnings, other long-term capital, and short-term capital as shown in the balance of payments. This series shows net inflows (new investment inflows less disinvestment) in the reporting economy from foreign investors, and is divided by GDP.
Source
International Monetary Fund, International Financial Statistics and Balance of Payments databases, World Bank, International Debt Statistics, and World Bank and OECD GDP estimates.

In [249]:
def get_countries(df, countries_df, iso="ISO3 Code", iso_data_name="Economy ISO3"):
    try:
        # Ensure the required column exists in the input DataFrame
        if iso_data_name not in df.columns:
            raise KeyError(f"Column '{iso_data_name}' not found in DataFrame")
        
        if iso not in countries_df.columns:
            raise KeyError(f"Column '{iso}' not found in countries DataFrame")
        
        # Process the ISO codes and clean whitespace
        iso_europe = set(countries_df[iso].dropna().astype(str).str.strip())
        iso_data = set(df[iso_data_name].dropna().astype(str).str.strip())
        
        # Print missing countries for debugging
        missing_countries = iso_data - iso_europe
        if missing_countries:
            print("Missing countries:", missing_countries)
        
        # Filter the DataFrame based on valid ISO codes
        filtered_df = df[df[iso_data_name].isin(iso_europe)]
        
        return filtered_df
    
    except KeyError as err:
        print(f"KeyError: {err}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [277]:
fdi = pd.read_csv("../data/raw/fdi.csv")

In [164]:
#fdi.columns = fdi.columns.str.strip().str.replace("\n", "", regex=True)


In [278]:
fdi = get_countries(df=fdi, countries_df=countries, iso_data_name="Country Code")

Missing countries: {'FRO', 'TJK', 'UZB', 'KGZ', 'AZE', 'KAZ', 'GIB', 'ARM', 'CHI', 'TKM', 'IMN', 'GRL'}


In [279]:
def convert_worldbank_df(df, value_name):
    
    df =df.drop(labels=["Series Code", "Series Name"], axis= 1)

    #df = df.drop(df.tail(8).index)

    df.columns = df.columns.str.replace(r"\[.*?\]", "", regex=True).str.strip()
    
    df = df.melt(
    id_vars=['Country Name', 'Country Code'],
    var_name='Year',
    value_name=value_name
)
    
    df.reset_index(drop=True, inplace=True)

    
    return df

In [280]:
fdi = convert_worldbank_df(df=fdi, value_name="FDI")

In [281]:
fdi.head()

,Country Name,Country Code,Year,FDI
0,Albania,ALB,2012,7.451355
1,Andorra,AND,2012,NaN
2,Austria,AUT,2012,1.283003
3,Belarus,BLR,2012,2.228180
4,Belgium,BEL,2012,2.369569


In [282]:
fdi["Country Code"].nunique()

47

### Unemployment Rate

Unemployment, total (% of total labor force)
(national estimate)
Unemployment refers to the share of the labor force that is without work but available for and seeking employment. Definitions of labor force and unemployment differ by country.

In [283]:
unemployement = pd.read_csv('../data/raw/unemployement.csv')

In [284]:
unemployement = get_countries(df=unemployement, countries_df=countries, iso_data_name="Country Code")

Missing countries: {'GHA', 'GNQ', 'THA', 'GTM', 'ABW', 'CYM', 'BEN', 'FJI', 'FSM', 'VEN', 'WSM', 'MUS', 'LSO', 'BWA', 'COM', 'BTN', 'BHR', 'GRD', 'ARM', 'SAU', 'SYR', 'TGO', 'PAN', 'GUM', 'NZL', 'STP', 'MWI', 'ZWE', 'GUY', 'IMN', 'MOZ', 'IRN', 'PNG', 'MAC', 'AUS', 'LBN', 'SEN', 'MRT', 'TLS', 'KWT', 'GMB', 'SLE', 'CHN', 'COL', 'OMN', 'SYC', 'KGZ', 'NCL', 'PRK', 'NGA', 'CUB', 'TJK', 'URY', 'CIV', 'MAR', 'ISR', 'YEM', 'PYF', 'USA', 'BHS', 'PRI', 'CPV', 'LAO', 'BRA', 'ATG', 'SWZ', 'TKM', 'JAM', 'KHM', 'TCA', 'BMU', 'PAK', 'LBY', 'DZA', 'CUW', 'BLZ', 'AGO', 'HND', 'GRL', 'SDN', 'MAF', 'MNG', 'AZE', 'SGP', 'SLV', 'COG', 'HKG', 'PHL', 'PSE', 'LKA', 'KAZ', 'KNA', 'TTO', 'VIR', 'ZAF', 'HTI', 'ARG', 'CMR', 'GIB', 'TCD', 'RWA', 'SLB', 'COD', 'CAN', 'NRU', 'TUV', 'EGY', 'KIR', 'BRB', 'UGA', 'TUN', 'CHI', 'MLI', 'NPL', 'LBR', 'VCT', 'NAM', 'MDV', 'NIC', 'IND', 'TZA', 'BRN', 'BGD', 'ETH', 'CHL', 'DOM', 'MEX', 'JOR', 'MNP', 'ERI', 'PLW', 'LCA', 'SUR', 'QAT', 'NER', 'KEN', 'ECU', 'BOL', 'ARE', 'MMR', 

In [285]:
unemployement["Country Code"].nunique()

47

In [286]:
unemployement = convert_worldbank_df(df=unemployement, value_name="Unemployement_rate")

In [288]:
unemployement.head()

,Country Name,Country Code,Year,Unemployement_rate
0,Albania,ALB,2012,13.376
1,Andorra,AND,2012,NaN
2,Austria,AUT,2012,4.909
3,Belarus,BLR,2012,0.500
4,Belgium,BEL,2012,7.542


## Gini Coefficent

Gini index
The Gini index measures the extent to which the distribution of income or consumption among individuals or households within an economy deviates from a perfectly equal distribution. A Gini index of 0 represents perfect equality, while an index of 100 implies perfect inequality.

Gini index measures the extent to which the distribution of income (or, in some cases, consumption expenditure) among individuals or households within an economy deviates from a perfectly equal distribution. A Lorenz curve plots the cumulative percentages of total income received against the cumulative number of recipients, starting with the poorest individual or household. The Gini index measures the area between the Lorenz curve and a hypothetical line of absolute equality, expressed as a percentage of the maximum area under the line. Thus a Gini index of 0 represents perfect equality, while an index of 100 implies perfect inequality.

In [194]:
gini = pd.read_csv('../data/raw/gini.csv')

In [202]:
gini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  552 non-null    object 
 1   Country Code  552 non-null    object 
 2   Year          552 non-null    object 
 3   Gini          388 non-null    float64
dtypes: float64(1), object(3)
memory usage: 17.4+ KB


In [196]:
gini = get_countries(df=gini, countries_df=countries, iso_data_name="Country Code")

Missing countries: {'GHA', 'GNQ', 'THA', 'GTM', 'ABW', 'CYM', 'BEN', 'FJI', 'FSM', 'VEN', 'WSM', 'MUS', 'LSO', 'BWA', 'COM', 'BTN', 'BHR', 'GRD', 'ARM', 'SAU', 'SYR', 'TGO', 'PAN', 'XKX', 'GUM', 'NZL', 'STP', 'MWI', 'ZWE', 'GUY', 'IMN', 'MOZ', 'IRN', 'PNG', 'MAC', 'AUS', 'LBN', 'SEN', 'MRT', 'TLS', 'KWT', 'GMB', 'SLE', 'CHN', 'COL', 'OMN', 'SYC', 'KGZ', 'NCL', 'PRK', 'NGA', 'CUB', 'TJK', 'URY', 'CIV', 'MAR', 'ISR', 'YEM', 'PYF', 'USA', 'BHS', 'PRI', 'CPV', 'LAO', 'BRA', 'ATG', 'SWZ', 'TKM', 'JAM', 'KHM', 'TCA', 'BMU', 'PAK', 'LBY', 'DZA', 'CUW', 'BLZ', 'AGO', 'HND', 'GRL', 'SDN', 'MAF', 'MNG', 'AZE', 'SGP', 'SLV', 'COG', 'HKG', 'PHL', 'PSE', 'LKA', 'KAZ', 'KNA', 'TTO', 'VIR', 'ZAF', 'HTI', 'ARG', 'CMR', 'GIB', 'TCD', 'RWA', 'SLB', 'COD', 'CAN', 'NRU', 'TUV', 'EGY', 'KIR', 'BRB', 'UGA', 'TUN', 'CHI', 'MLI', 'NPL', 'LBR', 'VCT', 'NAM', 'MDV', 'NIC', 'IND', 'TZA', 'BRN', 'BGD', 'ETH', 'CHL', 'DOM', 'MEX', 'JOR', 'MNP', 'ERI', 'PLW', 'LCA', 'SUR', 'QAT', 'NER', 'KEN', 'ECU', 'BOL', 'ARE', 

In [197]:
gini = convert_worldbank_df(df=gini, value_name="Gini")

## Rule of Law

Rule of Law captures perceptions of the extent to which agents have confidence in and abide by the rules of society, and in particular the quality of contract enforcement, property rights, the police, and the courts, as well as the likelihood of crime and violence. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

In [289]:
rule_of_law = pd.read_csv("../data/raw/rule_of_law.csv")

In [290]:
rule_of_law.head()

,Series Name,Series Code,Country Name,Country Code,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Control of Corruption: Estimate,CC.EST,Albania,ALB,-0.586572,-0.545141,-0.472374,-0.481387,-0.546019,-0.564274,-0.573539,-0.577180,-0.408189,-0.332219
1,Control of Corruption: Estimate,CC.EST,Andorra,AND,1.220694,1.151729,1.159411,1.171781,1.178940,1.176189,1.265454,1.279687,1.270272,1.251356
2,Control of Corruption: Estimate,CC.EST,Armenia,ARM,-0.565447,-0.609747,-0.659795,-0.637015,-0.409369,-0.227040,-0.003475,0.048685,0.027907,0.058400
3,Control of Corruption: Estimate,CC.EST,Austria,AUT,1.466502,1.472044,1.496881,1.500820,1.568587,1.521324,1.477709,1.242727,1.258587,1.133653
4,Control of Corruption: Estimate,CC.EST,Azerbaijan,AZE,-1.022828,-0.952573,-0.852637,-0.906533,-0.853234,-0.849682,-1.078076,-0.847890,-1.040907,-1.195471


In [291]:
rule_of_law = get_countries(df=rule_of_law, countries_df=countries, iso_data_name="Country Code")

Missing countries: {'JEY', 'TJK', 'UZB', 'KGZ', 'AZE', 'KAZ', 'ARM', 'TKM', 'GRL'}


In [244]:
rule_of_law = convert_worldbank_df(df=rule_of_law, value_name="Rule_of_law")

In [245]:
rule_of_law["Country Code"].unique()

array(['ALB', 'AND', 'AUT', 'BLR', 'BEL', 'BIH', 'BGR', 'HRV', 'CYP',
       'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'GEO', 'DEU', 'GRC', 'HUN',
       'IRL', 'ITA', 'XKX', 'LVA', 'LIE', 'LTU', 'LUX', 'MDA', 'MCO',
       'MNE', 'NLD', 'MKD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SMR',
       'SRB', 'SVK', 'SVN', 'ESP', 'SWE', 'CHE', 'TUR', 'UKR', 'GBR',
       'ISL', 'MLT'], dtype=object)

In [246]:
iso_europe = set(countries["ISO3 Code"].dropna().astype(str).str.strip())
iso_data = set(rule_of_law["Country Code"].dropna().astype(str).str.strip())

In [256]:
iso_europe-iso_data

{'RKS', 'VAT'}

So as the process would be the same for every dataset of the worldbank we created a loop and combined the end results to have a better overview in the code. You will find a overall description of every data point after the code.

Political Stability and Absence of Violence/Terrorism: Estimate
Political Stability and Absence of
Violence/Terrorism measures perceptions of the likelihood of political instability and/or politically-motivated violence, including terrorism. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

Control of Corruption: Estimate
Control of Corruption captures perceptions of the extent to which public power is exercised for private gain, including both petty and grand forms of corruption, as well as "capture" of the state by elites and private interests. Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately
-2.5 to 2.5.

GE.EST
Government Effectiveness: Estimate
Government Effectiveness captures perceptions of the quality of public services, the quality of the civil service and the degree of its independence from political pressures, the quality of policy formulation and implementation, and the credibility of the government's commitment to such policies.
Estimate gives the country's score on the aggregate indicator, in units of a standard normal distribution, i.e. ranging from approximately -2.5 to 2.5.

In [312]:
import os

In [258]:
rule_of_law = pd.read_csv("../data/raw/rule_of_law.csv")

In [326]:
# Define your datasets info
datasets_info = {
    "rule_of_law": {
        "file_path": "rule_of_law",
        "value_name": "Rule_of_law"
    },
    "government_effectiveness": {
        "file_path": "government_effectiveness",
        "value_name": "Gov_effectiveness"
    },
    "control_of_corruption": {
        "file_path": "control_of_corruption",
        "value_name": "Control_of_corruption"
    },
    "fdi": {
        "file_path": "fdi",
        "value_name": "Fdi"
    },
    "gdp_per_capita": {
        "file_path": "gdp_per_capita",
        "value_name": "Gdp_per_capita"
    },
    "gini": {
        "file_path": "gini",
        "value_name": "Gini"
    },
    "unemployement": {
        "file_path": "unemployement",
        "value_name": "Unemployement"
    },
}

# Suppose 'countries' is your DataFrame of valid countries
# For example: countries = pd.read_csv('../data/reference/countries.csv')




In [324]:
import os
import pandas as pd

def process_dataset(file_path, value_name, countries_df):
    """
    Reads a CSV file and processes it into a standardized format:
      - Reads the CSV
      - Converts it using 'convert_worldbank_df' (user-defined)
      - Matches and filters countries using 'get_countries' (user-defined)
      - Keeps only [Country Name, Country Code, Year, <value_name>]

    Parameters:
        file_path (str): Path to the CSV file (without the .csv extension).
        value_name (str): Name to be assigned to the indicator value column.
        countries_df (pd.DataFrame): A DataFrame containing valid country information
                                     for filtering/mapping.

    Returns:
        pd.DataFrame: Processed DataFrame with columns:
                      ['Country Name', 'Country Code', 'Year', <value_name>]
    """
    full_path = f'../data/raw/{file_path}.csv'
    
    # Check if file exists
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"File not found: {full_path}")
    
    # Read CSV
    df = pd.read_csv(full_path)
    
    # Convert the World Bank style data to a long format with 'Year' and 'value_name'
    df = convert_worldbank_df(df=df, value_name=value_name)
    
    # Filter/match country codes and names against your countries master table
    df = get_countries(df=df, countries_df=countries_df, iso_data_name="Country Code")

    # Keep only necessary columns
    # If you also want to keep 'Country Name' in the final DataFrame, 
    # include it here. Otherwise, omit it if you only want Code/Year.
    df = df[['Country Name', 'Country Code', 'Year', value_name]]
    
    return df


In [325]:
def combine_datasets(datasets_info, countries_df):
    """
    Iterates over all datasets in datasets_info, processes each one,
    and merges them into a single DataFrame on ['Country Code', 'Year'].

    Parameters:
        datasets_info (dict): Dictionary containing dataset info with keys like:
                              {
                                  "rule_of_law": {
                                      "file_path": "rule_of_law",
                                      "value_name": "Rule_of_law"
                                  },
                                  ... 
                              }
        countries_df (pd.DataFrame): A DataFrame containing valid country information.

    Returns:
        pd.DataFrame: Final merged DataFrame with columns:
                      ['Country Name', 'Country Code', 'Year', <all value_names>]
    """
    merged_df = pd.DataFrame()

    for ds_name, info in datasets_info.items():
        file_path = info["file_path"]
        value_name = info["value_name"]
        
        try:
            # Process each dataset
            temp_df = process_dataset(
                file_path=file_path,
                value_name=value_name,
                countries_df=countries_df
            )
            print(f"Successfully processed: {ds_name}")

            # Merge into our final DF
            if merged_df.empty:
                # If it's the first dataset, just assign it directly
                merged_df = temp_df
            else:
                # Merge on ['Country Code', 'Year'] (and optionally 'Country Name' if consistent)
                merged_df = pd.merge(
                    merged_df,
                    temp_df,
                    on=['Country Code', 'Year'],
                    how='outer'
                )
                
                # If you want only one 'Country Name' column without duplication,
                # you can do a small cleanup. E.g., if both sides have 'Country Name',
                # you might need to drop one or combine them if they differ.

        except Exception as e:
            print(f"Error processing {ds_name}: {e}")
            continue

    return merged_df


In [328]:
# Combine them all
final_df = combine_datasets(datasets_info, countries)

# Inspect the merged DataFrame
print(final_df.head())
print(final_df.columns)

Missing countries: {'JEY', 'TJK', 'UZB', 'KGZ', 'AZE', 'KAZ', 'ARM', 'TKM', 'GRL'}
Successfully processed: rule_of_law
Missing countries: {'JEY', 'TJK', 'UZB', 'KGZ', 'AZE', 'KAZ', 'ARM', 'TKM', 'GRL'}
Successfully processed: government_effectiveness
Missing countries: {'JEY', 'TJK', 'UZB', 'KGZ', 'AZE', 'KAZ', 'ARM', 'TKM', 'GRL'}
Successfully processed: control_of_corruption
Missing countries: {'FRO', 'TJK', 'UZB', 'KGZ', 'AZE', 'KAZ', 'GIB', 'ARM', 'CHI', 'TKM', 'IMN', 'GRL'}
Successfully processed: fdi
Error processing fdi: Passing 'suffixes' which cause duplicate columns {'Country Name_x'} is not allowed.
Missing countries: {'AZE', 'GIB', '(Europe)'}
Successfully processed: gdp_per_capita
Error processing gdp_per_capita: Passing 'suffixes' which cause duplicate columns {'Country Name_x'} is not allowed.
Missing countries: {'GHA', 'GNQ', 'THA', 'GTM', 'ABW', 'CYM', 'BEN', 'FJI', 'FSM', 'VEN', 'WSM', 'MUS', 'LSO', 'BWA', 'COM', 'BTN', 'BHR', 'GRD', 'ARM', 'SAU', 'SYR', 'TGO', 'PAN',

In [332]:
final_df.head()

,Country Name_x,Country Code,Year,Rule_of_law,Country Name_y,Gov_effectiveness,Country Name,Control_of_corruption
0,NaN,ALB,2012,NaN,Albania,-0.268925,Albania,-0.778729
1,NaN,ALB,2013,NaN,Albania,-0.324457,Albania,-0.751373
2,Albania,ALB,2014,-0.586572,Albania,-0.049771,Albania,-0.586572
3,Albania,ALB,2014,11.000000,Albania,-0.049771,Albania,-0.586572
4,Albania,ALB,2014,34.134617,Albania,-0.049771,Albania,-0.586572


In [334]:
def process_dataset(file_path, value_name, countries_df):
    ...
    print(f"Shape of {value_name} before final subset: {df.shape}")
    print(f"Columns of {value_name} DataFrame: {df.columns.tolist()}")
    print(df[['Country Code', 'Year']].drop_duplicates().head(5))  # first few rows
    ...
    return df
